In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pickle
import json
from rich import print

import warnings
warnings.filterwarnings('ignore')


In [9]:
# Condif With Feature names
with open('src/config.json', 'r') as f:
    config_dict = json.load(f)

y_column = config_dict['y_column']
numeric_features = config_dict['numeric_features']
categorical_features = config_dict['categorical_features']
all_features = numeric_features + categorical_features

# Processed Data
with open('data/processed/ml_data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train']
X_test = data_dict['X_test']
y_train = data_dict['y_train']
y_test = data_dict['y_test']
feature_names = data_dict['feature_names']
X_preprocessor = data_dict['X_preprocessor']
y_preprocessor = data_dict['y_preprocessor']

In [10]:
X_train

array([[-0.01112705, -0.01112705, -0.01047677, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01234849, -0.01234849, -0.01314327, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01366211, -0.01366211, -0.01651414, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.01262504, -0.01262504, -0.01374701, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01262504, -0.01262504, -0.01374701, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01062004, -0.01062004, -0.00936992, ...,  0.        ,
         0.        ,  0.        ]], shape=(9148, 38))

In [11]:
X_test

array([[-0.00937555, -0.00937555, -0.0066531 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01343165, -0.01343165, -0.01550791, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00578037, -0.00578037, -0.01651414, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.01163406, -0.01163406, -0.01158362, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01262504, -0.01262504, -0.01651414, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00985952, -0.00985952, -0.00770964, ...,  0.        ,
         0.        ,  0.        ]], shape=(2287, 38))

### Entrenamiento

In [12]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet

from src.evaluate import train_eval_model
from src.training import (
    optimize_hyperparameters,
    objective_random_forest,
    objective_lightgbm,
    objective_xgboost,
    objective_elastic_net
) 

models = []
scores = []
print('Tunning Random Forest')
best_params, score = optimize_hyperparameters(objective_random_forest, X_train, y_train.ravel())

models.append(RandomForestRegressor(**best_params))
scores.append(score)

print()
print('Tunning XGBoost')
best_params, score = optimize_hyperparameters(objective_xgboost, X_train, y_train.ravel())
models.append(XGBRegressor(**best_params))
scores.append(score)

print()
print('Tunning LGBoost')
best_params, score = optimize_hyperparameters(objective_lightgbm, X_train, y_train.ravel())
models.append(LGBMRegressor(**best_params))
scores.append(score)

print()
print('Tunning Elastic Net')
best_params, score = optimize_hyperparameters(objective_elastic_net, X_train, y_train.ravel())
models.append(ElasticNet(**best_params))
scores.append(score)

train_eval_model(models, scores, X_train, y_train, X_test, y_test, y_preprocessor = y_preprocessor)


Tunning Random Forest

Best trial: 85. Best value: -0.253546: 100%|██████████| 100/100 [12:02<00:00,  7.22s/it]

 Best score : -0.25354566377080856


Tunning XGBoost

Best trial: 76. Best value: -0.232845: 100%|██████████| 100/100 [02:42<00:00,  1.63s/it]

...

Best trial: 54. Best value: -0.242407: 100%|██████████| 100/100 [04:44<00:00,  2.85s/it]

 Best score : -0.24240729355108231


Tunning Elastic Net

Best trial: 4. Best value: -0.443049: 100%|██████████| 100/100 [00:11<00:00,  8.93it/s]

 Best score : -0.44304917359295637
==================== Best model ====================
ElasticNet(alpha=0.0029916982921842443, l1_ratio=0.6158491851758152,
           max_iter=1194, tol=1.4202183130417666e-05)


REGRESSION METRICS
MAE: 46654717.2636
R²: -0.0005
MAPE: 0.4846
Explained Variance: -0.0000
